In [1]:
import cupy as np # Use Cuda
from h5py import File
from PIL import Image
from sandbox import model, layers, activations, costs, predictions

np.random.seed(1)

In [2]:
# Load data
train_dataset = File('dataset\\train_catvnoncat.h5', 'r')
train_x = np.array(train_dataset['train_set_x'][:]) # Train set features
train_y = np.array(train_dataset['train_set_y'][:]) # Train set labels

test_dataset = File('dataset\\test_catvnoncat.h5', 'r')
test_x = np.array(test_dataset['test_set_x'][:]) # Test set features
test_y = np.array(test_dataset['test_set_y'][:]) # Test set labels

train_y = train_y.reshape((train_y.shape[0], 1))
test_y = test_y.reshape((test_y.shape[0], 1))

# Flatten and normalize
train_x = train_x.reshape(train_x.shape[0], -1) / 255
test_x = test_x.reshape(test_x.shape[0], -1) / 255

In [3]:
cat = model.Model(cuda=True)
cat.add(layers.Dense(units=20, activation=activations.ReLU()))
cat.add(layers.Dense(units=7, activation=activations.ReLU()))
cat.add(layers.Dense(units=5, activation=activations.ReLU()))
cat.add(layers.Dense(units=1, activation=activations.Sigmoid()))

cat.configure(learning_rate=0.0075, epochs=1500, cost_type=costs.BinaryCrossentropy())
cat.train(train_x, train_y, verbose=True)
cat.save() # Save model parameters

Cost on epoch 0: 0.68323
Cost on epoch 100: 0.58975
Cost on epoch 200: 0.50356
Cost on epoch 300: 0.48155
Cost on epoch 400: 0.42944
Cost on epoch 500: 0.39015
Cost on epoch 600: 0.33552
Cost on epoch 700: 0.30419
Cost on epoch 800: 0.49685
Cost on epoch 900: 0.41558
Cost on epoch 1000: 0.08405
Cost on epoch 1100: 0.05911
Cost on epoch 1200: 0.04401
Cost on epoch 1300: 0.03469
Cost on epoch 1400: 0.02821
Cost on epoch 1500: 0.02377


In [6]:
# Assess model accuracy
pred_train = cat.predict(train_x, prediction_type=predictions.binary_classification) # Get model accuracy on training data
print('\nTraining Accuracy: '  + str(np.round(np.sum((pred_train == train_y)/train_x.shape[0]), decimals=5)))
pred_test = cat.predict(test_x, prediction_type=predictions.binary_classification) # Get model accuracy on testing data
print('Testing Accuracy: '  + str(np.round(np.sum((pred_test == test_y)/test_x.shape[0]), decimals=5)))

# Use model on custom image
my_image = 'dataset\\not_cat.jpg' 
num_px = 64

image = np.array(Image.open(my_image).resize((num_px, num_px))) / 255 # Resize and normalize image, cast to NumPy array
image = image.reshape((1, num_px * num_px * 3)) # Flatten image array
my_predicted_image = cat.predict(image, prediction_type=predictions.binary_classification) # Predict custom image
print(f"Image guess: {my_predicted_image.item()}")


Training Accuracy: 0.99043
Testing Accuracy: 0.7
Image guess: 0
